In [1]:
# '''
# Author: Nischay Gowda nischaygowda105@gmail.com
# Date: 2023-02-08 12:17:30
# LastEditors: Nischay Gowda nischaygowda105@gmail.com
# LastEditTime: 2023-02-08 12:23:11
# '''

# PRAW package
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
import pandas as pd
from pmaw import PushshiftAPI
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import requests
import sqlite3
import datetime


import datetime, time
# importing timezone from pytz module
from pytz import timezone


c:\Users\nisch\anaconda3\envs\peoject_2_subreddit\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:

# fetch data using reddit url.
def get_subreddit_data(SUBREDDIT, LIMIT, TIMEFRAME):
    # without Authemticator
    # try:
    #     api = PushshiftAPI()
    #     post_data = api.submission(subreddit = SUBREDDIT)
    # except:
    #     print('An Error Occured')
    # return post_data
    api = PushshiftAPI()
    post_data = api.search_submissions(subreddit = SUBREDDIT,limit = LIMIT, listing = 'top')
    post_list = [post for post in post_data]
    return post_list

def get_data_in_dataframe(r_data):
    
    # create a dataframe
    data_dict = defaultdict()
    
    for r_post in r_data:
        
        # Data points available for a reddit post.
        # ['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 
        # 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'top_awarded_type', 
        # 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'subreddit_type', 
        # 'ups', 'total_awards_received', 'media_embed', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 
        # 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 
        # 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 
        # 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 
        # 'domain', 'allow_live_comments', 'selftext_html', 'likes', 'suggested_sort', 'banned_at_utc', 'view_count', 'archived', 'no_follow', 
        # 'is_crosspostable', 'pinned', 'over_18', 'all_awardings', 'awarders', 'media_only', 'can_gild', 'spoiler', 'locked', 'author_flair_text', 
        # 'treatment_tags', 'visited', 'removed_by', 'num_reports', 'distinguished', 'subreddit_id', 'author_is_blocked', 'mod_reason_by', 'removal_reason', 
        # 'link_flair_background_color', 'id', 'is_robot_indexable', 'report_reasons', 'author', 'discussion_type', 'num_comments', 'send_replies', 'whitelist_status', 
        # 'contest_mode', 'mod_reports', 'author_patreon_flair', 'author_flair_text_color', 'permalink', 'parent_whitelist_status', 'stickied', 'url', 'subreddit_subscribers', 
        # 'created_utc', 'num_crossposts', 'media', 'is_video']
        
        data_dict[r_post['id']] = {'id':r_post['id'],'created_date_utc':r_post['created_utc'],'subreddit_id':r_post['subreddit_id'],
                                   'title':r_post['title'],'body':r_post['selftext'], 'url':r_post['url'],'score':r_post['score'],'comments':r_post['num_comments']}
    
    main_df = pd.DataFrame.from_dict(data_dict, orient='index')
    return main_df
    
# def get_post_titles(r):
#     '''
#     Get a List of post titles
#     '''
#     posts = []
#     for post in r['data']['children']:
#         for val in post['data']:
#             posts.append(val)
#     return posts
    

In [6]:
# input parameters.
SUBREDDIT = 'askreddit'
LIMIT = 1000
TIMEFRAME = 'all'
LISTING = 'top'

# database
DATABASE_LOCATION = "sqlite:///subreddit_data.sqlite"

if __name__ == '__main__':
    r = get_subreddit_data(SUBREDDIT, LIMIT, TIMEFRAME)
    # print(r)
    
    main_data = get_data_in_dataframe(r)
    # print(main_data)
    # print(main_data.info())
    
    
    # giving the format of datetime
    format = "%Y-%m-%d"

    main_data['created_date'] = ''
    # convert float UTC to Date.
    for i in range(len(main_data)):
        main_data['created_date'][i] = datetime.datetime.fromtimestamp(main_data['created_date_utc'][i]).strftime(format)
    main_data.reset_index(inplace=True)
    main_data = main_data[['id','created_date_utc', 'created_date', 'subreddit_id', 'title', 'body', 'url','score', 'comments']]
    # print(main_data.size)
    
    # Loading to database
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('subreddit_data.sqlite')
    cursor = conn.cursor()
    
    # SQL query to create subreddit data.
    sql_query = ''' 
                CREATE TABLE IF NOT EXISTS subreddit_data(
                    ID VARCHAR(200),
                    CREATED_DATE_UTC VARCHAR(200),
                    CREATED_DATE VARCHAR(200),
                    SUBREDDIT_ID VARCHAR(200),
                    POST_TITLE VARCHAR(200),
                    POST_BODY VARCHAR(200),
                    POST_URL VARCHAR(200),
                    POST_KARMA INTEGER, 
                    POST_COMMENTS VARCHAR(200)                
                )
                '''
                
    cursor.execute(sql_query)
    print('Opened DB successful')
    
    try:
        main_data.to_sql('subreddit_data', engine, index = False, if_exists = 'replace')
    except:
        print('Data alredy exists')


Opened DB successful
